<a href="https://colab.research.google.com/github/afsheen8433/world_population_analysis/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing kaggle library
! pip install kaggle

Uploading json file of kaggle.

In [ ]:
#configuring path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

now we will import the twitter sentiment dataset

In [ ]:
!kaggle datasets download -d Kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/Kazanova/sentiment140
License(s): other


In [ ]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


importing the dependencies
pandas, numpy
re regular expression - pattern matching and search through the data

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #for stemming processing that we do to reduce the words to root words
from sklearn.feature_extraction.text import TfidfVectorizer #covert the textual data to vectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

stopwords that are required for processing. need to remov these


In [ ]:
#printing stopwords in english

print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing
loading data


In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
#number of rowa and columns
twitter_data.shape

(1599999, 6)

In [ ]:
#print 5 rows
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#renaming the columns
column_names = ['target','id','date','flag','user','text']

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#missing values in dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


if distribution is not even then the model will not work properly

In [ ]:
#chekcing the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
#convert 4 to 1
twitter_data['target']=twitter_data['target'].replace(4,1)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


**Stemming**

it is the process of reducing words to there root words


In [ ]:
port_stem =PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content =re.sub('[^a-zA-Z]',' ', content)
  steemed_content= stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer you...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dive mani time ball manag save the ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad i see


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer you...
1          upset updat facebook text might cri result sch...
2          kenichan i dive mani time ball manag save the ...
3                            whole bodi feel itchi like fire
4                            nationwideclass behav mad i see
                                 ...                        
1599995                 just woke have school best feel ever
1599996    thewdb com veri cool hear old walt interview h...
1599997                     are readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer you shoulda got david carr third day d'
 'upset updat facebook text might cri result school today also blah'
 'kenichan i dive mani time ball manag save the rest go bound' ...
 'are readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y
      )

[0 0 0 ... 1 1 1]


In [ ]:
X_train,X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin i'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day can wait hear guy store tomorrow'
 'i love wake folger too bad voic deeper']


In [ ]:
print(X_test)

['mmangen m fine i much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'not feel well'
 'supersandro thank']


In [ ]:
vectorizer = TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9945054 stored elements and shape (1280000, 461516)>
  Coords	Values
  (0, 436736)	0.27259876264838384
  (0, 354563)	0.3588091611460021
  (0, 185208)	0.5277679060576009
  (0, 109311)	0.3753708587402299
  (0, 235060)	0.41996827700291095
  (0, 443091)	0.4484755317023172
  (1, 160645)	1.0
  (2, 109311)	0.45915639201149566
  (2, 124490)	0.18923156633380322
  (2, 407323)	0.18710917798380342
  (2, 129417)	0.29076646660098543
  (2, 406421)	0.32108169269258585
  (2, 433582)	0.32968741386905526
  (2, 77931)	0.3128672120237835
  (2, 443455)	0.33488823001589063
  (2, 266745)	0.2412526672795364
  (2, 409165)	0.15170562659204556
  (2, 178072)	0.16191467578116914
  (2, 150722)	0.18805437673647912
  (2, 132317)	0.20291428205045584
  (2, 288488)	0.1673801447945276
  (3, 406421)	0.29029991238662284
  (3, 158719)	0.4456939372299574
  (3, 151777)	0.278559647704793
  (3, 56478)	0.5200465453608686
  :	:
  (1279996, 390152)	0.2206474219107611
  (

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2412945 stored elements and shape (320000, 461516)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31169)	0.1624772418052177
  (0, 67830)	0.26800375270827315
  (0, 106074)	0.36555450010904555
  (0, 132370)	0.255254889555786
  (0, 138171)	0.23688292264071406
  (0, 171389)	0.2805816206356074
  (0, 271032)	0.45356623916588285
  (0, 279098)	0.17825180109103442
  (0, 388370)	0.2198507607206174
  (0, 398928)	0.34910438732642673
  (0, 409165)	0.3143047059807971
  (0, 421006)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145400)	0.575262969264869
  (1, 217577)	0.40288153995289894
  (1, 256792)	0.28751585696559306
  (1, 348155)	0.4739279595416274
  (1, 366225)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34402)	0.37916255084357414
  (2, 89451)	0.36340369428387626
  (2, 183325)	0.5892069252021465
  (2, 256849)	0.2564939661498776
  :	:
  (319995, 213339)	0.24848254898331648

training the ml model

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

model evaluation accuracy score

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score on the training data:',training_data_accuracy)

Accuracy score on the training data: 0.8020359375


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score on the test data:',test_data_accuracy)

Accuracy score on the test data: 0.780575


In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model = pickle.load(open('trained_model.sav','rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])
prediction = model.predict(X_new)
print(prediction)
if(prediction[0]==0):
 print('negative tweet')
else:
  print('positive tweet')

1
[1]
positive tweet


In [ ]:
X_new = X_test[3]
print(Y_test[3])
prediction = model.predict(X_new)
print(prediction)
if(prediction[0]==0):
 print('negative tweet')
else:
  print('positive tweet')

0
[0]
negative tweet
